In [1]:
import pandas as p
import numpy as n

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [3]:
data=p.read_csv(r'C:\Users\SOHAIL SONS TRADERS\Downloads\py-master\py-master\ML\14_naive_bayes\titanic.csv')
data.head(3)

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1


In [ ]:
df=p.DataFrame(data)

In [ ]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis='columns',inplace=True)

In [ ]:
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'],
                                                    test_size=0.2,
                                                   random_state=42)

In [ ]:
y_test

In [ ]:
df.isnull().sum()

In [ ]:
trf1=ColumnTransformer([
    ('compute_age',SimpleImputer(),[2]),
    ('compute_embark',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [ ]:
trf2=ColumnTransformer([
    ('ohe_sex_embark',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [ ]:
trf3=ColumnTransformer([
    ('scal',MinMaxScaler(),slice(0,10))
])

In [ ]:
trf4=SelectKBest(score_func=chi2,k=8)

In [ ]:
trf5=DecisionTreeClassifier()


# CREAT PIPLINE

In [ ]:
pip=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [ ]:
pip.fit(X_train,y_train)

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
pip

In [ ]:
pip.named_steps['trf1'].transformers_[0][1].statistics_

In [ ]:
pip.named_steps['trf2']

In [ ]:
y_pre=pip.predict(X_test)

In [ ]:
y_pre

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pre)

# USING CROSS VALIDATION

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pip,X_train,y_train,cv=5).mean()

# USING GridSearchCV

In [ ]:
params={
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pip,params,cv=5)
grid.fit(X_train,y_train)

In [ ]:
grid.best_estimator_

# EXPORTING THE PIPLINE

In [ ]:
import pickle
pickle.dump(pip,open('pickel.pkl','wb'))